# Import Library

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('data_scraping.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,9c6863cf-0390-467c-8899-741375e562f6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,INDODAX 3.0 cocok banget buat pemula seperti s...,5,9,5.4.8,2025-04-05 14:23:04,"Dear member, terima kasih telah mengikuti even...",2025-04-05 14:26:08,5.4.8
1,394f16ba-7be1-4aa0-bdec-5d6be1e359bb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi nya mudah diakses dan digunakan, terp...",5,94,5.4.8,2025-03-20 12:11:49,"Dear member INDODAX, terima kasih atas review ...",2025-03-20 12:55:39,5.4.8
2,b234ca11-9cc0-4d86-81ad-fb44abff9e7e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi Indodax sangat user-friendly, ideal u...",5,9,5.4.8,2025-04-05 17:26:48,"Dear member, terima kasih telah mengikuti even...",2025-04-06 01:37:22,5.4.8
3,5b28bc72-d4e5-43e3-b149-718ad6cb2349,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Tidak diragukan lagi, INDODAX adalah salah sat...",5,1,5.4.8,2025-04-05 17:34:33,"Dear member, terima kasih telah mengikuti even...",2025-04-06 01:37:30,5.4.8
4,1b84f25e-317c-4748-bc8b-fb2de4c20102,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"indodax aneh akun tiba² terkunci, pas pelapora...",1,3,5.4.8,2025-04-05 00:15:54,Dear member INDODAX. Mohon maaf atas ketidakny...,2025-04-05 02:36:09,5.4.8


# Data Pre-Processing

In [4]:
data_null = df.isnull().sum()
data_duplicate = df.duplicated().sum()
print(f"Null values in each column:\n{data_null}")
print(f"Duplicate values in each column:\n{data_duplicate}")

Null values in each column:
reviewId                    0
userName                    0
userImage                   0
content                     0
score                       0
thumbsUpCount               0
reviewCreatedVersion    10493
at                          0
replyContent                0
repliedAt                   0
appVersion              10493
dtype: int64
Duplicate values in each column:
0


**Insight**
- Terdapat **10493** baris data yang bernilai null/nan pada kolom **reviewCreatedVersion** dan kolom **appVersion**
- Data yang bernilai null/nan pada data tersebut sebenarnya tidak terlalu mempengaruhi proses pelatihan karena kolom yang bernilai null tidak akan digunakan pada proses analisis sentimen sehingga saya tidak akan menghapus data tersebut karena data tersebut cukup banyak dan akan mereduksi banyak sekali data

# Text Pre-Processing

In [ ]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = re.sub(r'[⁰¹²³⁴⁵⁶⁷⁸⁹]', '', text)

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','bagu', 'sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy", "udh", "sudah", "tidak", "indodax", "aplikasi", "exchange", "udah"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def removeShortWords(text, length=3):
    # Menghapus kata yang panjangnya tepat sesuai parameter length (default = 3)
    words = text.split()
    filtered_words = [word for word in words if len(word) != length]
    return ' '.join(filtered_words)

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "udh" : "sudah", "yg" : "yang"}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text